In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.datasets import dump_svmlight_file


import boto3
import sagemaker.amazon.common as smac

<h2>Movie Recommendations Dataset</h2>
<h4>Hands-on: Classification with SageMaker</h4>
Input Features: [userId, moveId] <br>
Target Feature: rating <br>
Objective: Predict how a user would rate a particular movie<br>
<h4>Movie Lens Overview: https://grouplens.org/datasets/movielens/</h4>
<h4>Dataset: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip</h4>
<h4>F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. </h4>

<h3>Load Movies and Parse Genre</h3>

#### Download Movie Dataset from grouplens

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [ ]:
ls

In [ ]:
!unzip ml-latest-small.zip

In [ ]:
df_movies = pd.read_csv(r'ml-latest-small/movies.csv')

In [ ]:
df_movies.shape

In [ ]:
df_movies.head()

In [ ]:
genre_list = df_movies.genres.map(lambda value: value.split('|'))

In [ ]:
genre_list[:10]

In [ ]:
def get_unique_genres (genre_list):
    unique_list = set()
    
    for items in genre_list:
        for item in items:
            unique_list.add(item)
    
    return sorted(unique_list)

In [ ]:
genre = get_unique_genres(genre_list)

In [ ]:
genre, len(genre)

In [ ]:
# Table of genre for each movie
df_genre = pd.DataFrame(index=range(df_movies.shape[0]),columns=genre)

In [ ]:
df_genre = df_genre.fillna(0)

In [ ]:
df_genre.shape

In [ ]:
df_genre.head()

In [ ]:
genre_list [:11]

In [ ]:
# Fill genre for each movie
for row, movie_genre in enumerate(genre_list):
    df_genre.loc[row][movie_genre] = 1

In [ ]:
df_genre.head()

In [ ]:
# Some movies don't have genre listed
df_genre[df_genre['(no genres listed)'] > 0].head()

In [ ]:
# Merge with movie description
df_movies = df_movies.join(df_genre)

In [ ]:
df_movies.head()

In [ ]:
df_movies.to_csv(r'ml-latest-small/movies_genre.csv', index=False)

<h3>Load Ratings given by each user for a movie</h3>

In [ ]:
df_ratings = pd.read_csv(r'ml-latest-small/ratings.csv')

In [ ]:
df_ratings.head()

In [ ]:
df_ratings.userId.unique().shape

In [ ]:
df_ratings.movieId.unique().shape

In [ ]:
df_ratings.drop(axis=1,columns=['timestamp'],inplace=True)

In [ ]:
# Merge rating and movie description
df_movie_ratings = pd.merge(df_ratings,df_movies,on='movieId')

In [ ]:
df_movie_ratings.head(2)

In [ ]:
df_movie_ratings.tail(2)

## Training and Validation Set
### Target Variable as first column followed by input features:

### Training, Validation files do not have a column header

In [ ]:
# Training = 70% of the data
# Validation = 30% of the data
# Randomize the datset
np.random.seed(5)
l = list(df_movie_ratings.index)
np.random.shuffle(l)
df = df_movie_ratings.iloc[l]

In [ ]:
rows = df.shape[0]
train = int(.7 * rows)
test = rows-train

In [ ]:
rows,train,test

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
# SageMaker Factorization Machine expects all columns to be of float32
# Let's get the target variable as float32
y = df['rating'].astype(np.float32).ravel()

In [ ]:
len(y)

In [ ]:
y.dtype

In [ ]:
# We will create two different training datasets.
# Training 1: rating, user id, movie id
# Training 2: rating, user id, movie id, and movie genre attributes
columns_user_movie = ['userId','movieId']
columns_all = columns_user_movie + genre

In [ ]:
columns_user_movie

In [ ]:
columns_all

In [ ]:
# Store a copy of user id, movie id and rating
# Train and Test
df[['rating','userId','movieId']][:train].to_csv(r'ml-latest-small/user_movie_train.csv', index=False)
df[['rating','userId','movieId']][train:].to_csv(r'ml-latest-small/user_movie_test.csv',index=False)

In [ ]:
# One Hot Encode
# Training 1: user id, movie id
# Training 2: user id, movie id, and movie genre attributes
encoder = preprocessing.OneHotEncoder(dtype=np.float32)

In [ ]:
X = encoder.fit_transform(df[columns_user_movie])

In [ ]:
df.userId.unique().shape, df.movieId.unique().shape

In [ ]:
# Write Dimensions - we need it for training and prediction
# Number of unique users and movies
dim_movie = df.userId.unique().shape[0] + df.movieId.unique().shape[0]
with open(r'ml-latest-small/movie_dimension.txt','w') as f:
    f.write(str(dim_movie))

In [ ]:
X

In [ ]:
X.shape[1]

In [ ]:
# Create a spare matrix recordio file
def write_sparse_recordio_file (filename, x, y=None):
    with open(filename, 'wb') as f:
        smac.write_spmatrix_to_sparse_tensor (f, x, y)

In [ ]:
# Training recordIO file
write_sparse_recordio_file(r'ml-latest-small/user_movie_train.recordio',X[:train],y[:train])

In [ ]:
# Test recordIO file
write_sparse_recordio_file(r'ml-latest-small/user_movie_test.recordio',X[train:],y[train:])

In [ ]:
# Create libSVM formatted file. Convenient text format
# Output is stored as rating, user_index:value, movie_index:value
#  For example: 5.0 314:1 215:1  (user with index 314 and movie with index 215 in the one hot encoded table has a rating of 5 )

# This file can be used for two purposes: 
#   1. directly traing with libFM binary in local mode
#   2. It is easy to run inference with this format against sagemaker cloud as we need to
#      send only sparse input to sagemaker prediction service

# 
# Store in libSVM format as well for directly testing with libFM
dump_svmlight_file(X[:train],y[:train],r'ml-latest-small/user_movie_train.svm')
dump_svmlight_file(X[train:],y[train:],r'ml-latest-small/user_movie_test.svm')

In [ ]:
# Create two lookup files
# File 1: Categorical Movie ID and corresponding Movie Index in One Hot Encoded Table
# File 2: Categorical User ID and corresponding User Index in One Hot Encoded Table

# This is useful for predicting how a particular user would rate all the movies
# or all users rating one particular movie

list_of_movies = df.movieId.unique()
# user 1 and all movies
df_user_movie = pd.DataFrame({'userId': np.full(len(list_of_movies),1), 'movieId' : list_of_movies})

In [ ]:
df_user_movie[columns_user_movie].head()

In [ ]:
list_of_movies

In [ ]:
# Transform to one hot encoding (with existing encoder)
X = encoder.transform(df_user_movie[columns_user_movie])

In [ ]:
# Store movieId and corresponding one hot encoded entries
dump_svmlight_file(X,list_of_movies,r'ml-latest-small/one_hot_enc_movies.svm')

In [ ]:
# Now create 
# File 2: Categorical User ID and corresponding User Index in One Hot Encoded Table
list_of_users = df.userId.unique()

In [ ]:
list_of_users.shape

In [ ]:
list_of_users[:10]

In [ ]:
# All users and movie 1
df_user_movie = pd.DataFrame({'userId': list_of_users, 'movieId' : np.full(len(list_of_users),1)})

In [ ]:
df_user_movie.head()

In [ ]:
# Transform to one hot encoding (with existing encoder)
X = encoder.transform(df_user_movie[columns_user_movie])

In [ ]:
# Store movieId and corresponding one hot encoded entries
dump_svmlight_file(X,list_of_users,r'ml-latest-small/one_hot_enc_users.svm')